In [ ]:
import os
import gc
import sys

import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import timm
from fastai.vision.all import Path, set_seed, Learner, BCEWithLogitsLossFlat, SaveModelCallback, EarlyStoppingCallback
from fastai.vision.all import ImageDataLoaders, RegressionBlock, Resize, Brightness, Contrast, Hue, Saturation, setup_aug_tfms

if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

In [ ]:
def base_func():
    seed = 999
    set_seed(seed, reproducible=True)
    device = 'cuda'
    BATCH_SIZE = 8                       #was 32
    N_FOLDS = 5         #was10
    PRED_FROM = 'pth'

    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True

    dataset_path = Path('../input/petfinder-pawpularity-score/')
    
    def get_train_df():
        train_df = pd.read_csv(dataset_path/'train.csv')
        train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
        train_df = train_df.drop(columns=['Id'])
        train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
        train_df['norm_score'] = train_df['Pawpularity']/100

        return train_df


    def kfold_train_df(train_df):
        #Sturges' rule
        num_bins = int(np.floor(1+(3.3)*(np.log2(len(train_df)))))
        # num_bins
        train_df['bins'] = pd.cut(train_df['norm_score'], bins=num_bins, labels=False)
        train_df['fold'] = -1

        strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
        for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
            train_df.iloc[train_index, -1] = i

        train_df['fold'] = train_df['fold'].astype('int')

        return train_df

    def petfinder_rmse(input,target):
        #return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))
        return 100*torch.sqrt(F.mse_loss(torch.sigmoid(input.flatten()), target))

    def get_data(train_df, fold):
        train_df_f = train_df.copy()
        # add is_valid for validation fold
        train_df_f['is_valid'] = (train_df_f['fold'] == fold)

        dls = ImageDataLoaders.from_df(train_df_f, #pass in train DataFrame
    #                                valid_pct=0.2, #80-20 train-validation random split
                                   valid_col='is_valid', #
                                   seed=seed, #seed
                                   fn_col='path', #filename/path is in the second column of the DataFrame
                                   label_col='norm_score', #label is in the first column of the DataFrame
                                   y_block=RegressionBlock, #The type of target
                                   bs=BATCH_SIZE, #pass in batch size
                                   num_workers=8,
                                   item_tfms=Resize(224), #pass in item_tfms
                                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) #pass in batch_tfms

        return dls


    def get_learner(train_df, fold_num):
        data = get_data(train_df, fold_num)

        model = timm.create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=data.c)

        learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()

        return learn

    def get_test_df():
        test_df = pd.read_csv(dataset_path/'test.csv')
        test_df['Pawpularity'] = [1]*len(test_df)
        test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
        test_df = test_df.drop(columns=['Id'])

        return test_df
    
    
    train_df = get_train_df()

    set_seed(seed, reproducible=True)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    train_df = kfold_train_df(train_df)

    #Valid Kfolder size
    the_data = get_data(train_df, 0)
    assert (len(the_data.train) + len(the_data.valid)) == (len(train_df)//BATCH_SIZE)

    test_df = get_test_df()

    #print(get_learner(train_df, fold_num=0).lr_find(end_lr=3e-2))             #was-2)

    all_preds = []
    for i in range(N_FOLDS):
        print(f'Fold {i} results')
        learn = get_learner(train_df, fold_num=i)

        if PRED_FROM == 'learning':
            learn.fit_one_cycle(5, 2e-5, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=2)]) 
            learn.recorder.plot_loss()

            #learn = learn.to_fp32()
            #learn.export(f'model_fold_{i}_export.pkl')
            learn.save(f'model_fold_{i}_save.pkl')
        elif PRED_FROM == 'pkl':
            learn.load(f'model_fold_{i}_save.pkl')
        elif PRED_FROM == 'pth':
            learn.model.load_state_dict(torch.load(f'../input/f-model0/model_fold_{i}_dict.pth'))
            learn.model.to(device)

        dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                                valid_pct=0.2, #80-20 train-validation random split
                                seed=seed, #seed
                                fn_col='path', #filename/path is in the second column of the DataFrame
                                label_col='norm_score', #label is in the first column of the DataFrame
                                y_block=RegressionBlock, #The type of target
                                bs=BATCH_SIZE, #pass in batch size
                                num_workers=8,
                                item_tfms=Resize(224), #pass in item_tfms
                                batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) 

        test_dl = dls.test_dl(test_df)
        preds, _ = learn.tta(dl=test_dl, n=5, beta=0)

        if PRED_FROM == 'learning':
            torch.save(learn.model.to('cpu').state_dict(), f'model_fold_{i}_dict.pth')

        all_preds.append(preds)
        del learn
        torch.cuda.empty_cache()
        gc.collect()

    preds = np.mean(np.stack(all_preds), axis=0)
    return preds

In [ ]:
preds1 = base_func().ravel()
print(preds1)

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
from fastai.vision.all import *

if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

import math

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

import gc

In [ ]:
def doggo_process():
    device = 'cuda'
    set_seed(365, reproducible=True)
    BATCH_SIZE = 32
    
    dataset_path = Path('../input/petfinder-pawpularity-score/')
    #dataset_path.ls()
    
    train_df = pd.read_csv(dataset_path/'train.csv')
    #train_df.head()
    
    train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
    train_df = train_df.drop(columns=['Id'])
    train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
    #train_df.head()
    
    len_df = len(train_df)
    print(f"There are {len_df} images")
    
    #train_df['Pawpularity'].hist(figsize = (10, 5))
    print(f"The mean Pawpularity score is {train_df['Pawpularity'].mean()}")
    print(f"The median Pawpularity score is {train_df['Pawpularity'].median()}")
    print(f"The standard deviation of the Pawpularity score is {train_df['Pawpularity'].std()}")
    
    print(f"There are {len(train_df['Pawpularity'].unique())} unique values of Pawpularity score")
    
    train_df['norm_score'] = train_df['Pawpularity']/100
    #train_df['norm_score']
    
    seed=365
    set_seed(seed, reproducible=True)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True
    
    #Rice rule
    num_bins = int(np.ceil(2*((len(train_df))**(1./3))))
    #num_bins
    
    train_df['bins'] = pd.cut(train_df['norm_score'], bins=num_bins, labels=False)
    #train_df['bins'].hist()
    
    train_df['fold'] = -1

    N_FOLDS = 10
    strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
    for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
        train_df.iloc[train_index, -1] = i

    train_df['fold'] = train_df['fold'].astype('int')

    #train_df.fold.value_counts().plot.bar()
    
    def petfinder_rmse(input,target):
        return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))
    
    def get_data(fold):
        #     train_df_no_val = train_df.query(f'fold != {fold}')
        #     train_df_val = train_df.query(f'fold == {fold}')

        #     train_df_bal = pd.concat([train_df_no_val,train_df_val.sample(frac=1).reset_index(drop=True)])
        train_df_f = train_df.copy()
        # add is_valid for validation fold
        train_df_f['is_valid'] = (train_df_f['fold'] == fold)

        dls = ImageDataLoaders.from_df(train_df_f, #pass in train DataFrame
        #                                valid_pct=0.2, #80-20 train-validation random split
                                   valid_col='is_valid', #
                                   seed=365, #seed
                                   fn_col='path', #filename/path is in the second column of the DataFrame
                                   label_col='norm_score', #label is in the first column of the DataFrame
                                   y_block=RegressionBlock, #The type of target
                                   bs=BATCH_SIZE, #pass in batch size
                                   num_workers=8,
                                   item_tfms=Resize(224), #pass in item_tfms
                                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) #pass in batch_tfms

        return dls
    
    #Valid Kfolder size
    the_data = get_data(0)
    assert (len(the_data.train) + len(the_data.valid)) == (len(train_df)//BATCH_SIZE)
    
    def get_learner(fold_num):
        data = get_data(fold_num)

        model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=data.c)

        learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()

        return learn
    
    test_df = pd.read_csv(dataset_path/'test.csv')
    #test_df.head()
    
    test_df['Pawpularity'] = [1]*len(test_df)
    test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
    test_df = test_df.drop(columns=['Id'])
    train_df['norm_score'] = train_df['Pawpularity']/100
    
    #get_learner(fold_num=0).lr_find(end_lr=3e-2)
    
    
    all_preds = []

    for i in range(N_FOLDS):

        print(f'Fold {i} results')

        learn = get_learner(fold_num=i)

        #learn.fit_one_cycle(5, 4e-5, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=2)]) 

        #learn.recorder.plot_loss()
        
        learn.model.load_state_dict(torch.load(f'../input/doggo-model2/model_fold_{i}_dict.pth'))
        learn.model.to(device)

        #learn = learn.to_fp32()

        #learn.export(f'model_fold_{i}.pkl')
        #learn.save(f'model_fold_{i}.pkl')

        dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                                   valid_pct=0.2, #80-20 train-validation random split
                                   seed=365, #seed
                                   fn_col='path', #filename/path is in the second column of the DataFrame
                                   label_col='norm_score', #label is in the first column of the DataFrame
                                   y_block=RegressionBlock, #The type of target
                                   bs=BATCH_SIZE, #pass in batch size
                                   num_workers=8,
                                   item_tfms=Resize(224), #pass in item_tfms
                                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) 

        test_dl = dls.test_dl(test_df)

        preds, _ = learn.tta(dl=test_dl, n=5, beta=0)

        all_preds.append(preds)
        
        #torch.save(learn.model.to('cpu').state_dict(), f'model_fold_{i}_dict.pth')

        del learn

        torch.cuda.empty_cache()

        gc.collect()

    #np.mean(np.stack(all_preds*100))
    
    
    preds = np.mean(np.stack(all_preds), axis=0)
    return preds

In [ ]:
preds2 = doggo_process().ravel()
print(preds2)

In [ ]:
import os
import gc
import sys

import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import StratifiedKFold

sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'
!cp '../input/efficientnet-pytorch-offline-pack/efficientnet-b7-dcc49843.pth' '/root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth'

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torchvision.transforms as T

import timm
from fastai.vision.all import Path, set_seed, DataLoader, DataLoaders, Learner, BCEWithLogitsLossFlat, SaveModelCallback, EarlyStoppingCallback
from fastai.vision.all import Brightness, Contrast, Hue, Saturation

from tqdm import tqdm
import argparse

BATCH_SIZE = 8
DENSE_FEATURE_COLS = [
    'dogcat_ohe0', 'dogcat_ohe1'
]
DATASET_PATH = Path('../input/petfinder-pawpularity-score/')
DOGCAT_MODEL_PATH = '../input/fohe-model0/sample_model.pth'

N_FOLDS = 5
BINS_RULE = 'sturge'

seed=999
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

device = 'cuda'
LOSS_TYPE = 'bce'
MODEL_INPUT = 'fp'
MODEL_HEAD = 'dense'
AFFINE_AUGMENTATION = False
RESIZE = 224
PRETRAINED = 'swin_large_patch4_window7_224'


#------- dog cat begin
sys.path.append('../input/efficientnet-pytorch-offline-pack/EfficientNet-PyTorch')
from efficientnet_pytorch import EfficientNet

def get_dogcat_model(model_path):
    model = EfficientNet.from_pretrained('efficientnet-b7')
    model._fc = nn.Linear(model._fc.in_features, 2)
    model.load_state_dict(torch.load(model_path))
    return model

class DogCatImageTransform():
    def __init__(self):
        resize = 150
        mean = (0.5,0.5,0.5)
        std = (0.5,0.5,0.5)
        self.data_transform = T.Compose([
            T.Resize((resize,resize)),
            T.ToTensor(),
            T.transforms.Normalize(mean, std)
        ])

    def __call__(self, img):
        return self.data_transform(img)

class DogCatDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
        self.transform = DogCatImageTransform()

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img = self.transform(img)

        return img

def calc_dogcat_prob(path_list, model_path):
    print(len(path_list))
    dogcat_dataset = DogCatDataset(path_list)
    dogcat_dataloader = DataLoader(dogcat_dataset, batch_size=64, shuffle=False)
    model = get_dogcat_model(model_path).to(device)
    model.eval()

    with torch.no_grad():
        preds = None
        for x in tqdm(dogcat_dataloader):
            output = model(x.to(device))
            output = torch.softmax(output, dim=1)
            output = output[:,0]
            preds = output if preds is None else torch.cat([preds, output], dim=0)

        return preds.flatten().cpu().numpy()

#------- dog cat end


def get_train_df(dataset_path, n_folds):
    train_df = pd.read_csv(dataset_path/'train.csv')
    train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
    train_df = train_df.drop(columns=['Id'])
    train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
    train_df['norm_score'] = train_df['Pawpularity']/100

    if BINS_RULE == 'sturge':
        num_bins = int(np.floor(1+(3.3)*(np.log2(len(train_df)))))
    elif BINS_RULE == 'rice':
        num_bins = int(np.ceil(2*((len(train_df))**(1./3))))
    train_df['bins'] = pd.cut(train_df['norm_score'], bins=num_bins, labels=False)
    train_df['fold'] = -1

    strat_kfold = StratifiedKFold(n_splits=n_folds, random_state=seed, shuffle=True)
    for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
        train_df.iloc[train_index, -1] = i
        
    train_df['fold'] = train_df['fold'].astype('int')

    if 'dogcat_ohe0' in DENSE_FEATURE_COLS:
        train_df['dogcat_prob'] = 0.0
        # モデルを読み込む場合は計算する必要が無いためコメントアウト
        #train_df['dogcat_prob'] = calc_dogcat_prob(train_df['path'].values, DOGCAT_MODEL_PATH)
        train_df['dogcat_ohe0'] = train_df['dogcat_prob'].apply(lambda x: 1 if x < 0.5 else 0)
        train_df['dogcat_ohe1'] = train_df['dogcat_prob'].apply(lambda x: 0 if x < 0.5 else 1)
        train_df = train_df.drop(columns=['dogcat_prob'])

    return train_df

def get_test_df(dataset_path):
    test_df = pd.read_csv(dataset_path/'test.csv')
    test_df['Pawpularity'] = [1]*len(test_df)
    test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
    test_df = test_df.drop(columns=['Id'])
    test_df['norm_score'] = test_df['Pawpularity']/100

    if 'dogcat_ohe0' in DENSE_FEATURE_COLS:
        test_df['dogcat_prob'] = calc_dogcat_prob(test_df['path'].values, DOGCAT_MODEL_PATH)
        test_df['dogcat_ohe0'] = test_df['dogcat_prob'].apply(lambda x: 1 if x < 0.5 else 0)
        test_df['dogcat_ohe1'] = test_df['dogcat_prob'].apply(lambda x: 0 if x < 0.5 else 1)
        test_df = test_df.drop(columns=['dogcat_prob'])

    return test_df


class PawpularDataset(Dataset):
    def __init__(self, image_paths, dense_features, targets, transform):
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        img_path = self.image_paths[item]
        img = Image.open(img_path)
        img_trans = self.transform(img)

        features = self.dense_features[item, :]
        targets = self.targets[item]

        return (img_trans, torch.tensor(features, dtype=torch.float)), torch.tensor(targets, dtype=torch.float)

class ImageTransform():
    def __init__(self):
        t_list = [T.Resize((RESIZE,RESIZE))]
        if AFFINE_AUGMENTATION:
            t_list.append(T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)))
        t_list.append(T.ToTensor())
        self.data_transform1 = T.Compose(t_list)

        self.data_transform2 = [Brightness(), Contrast(), Hue(), Saturation()]

    def __call__(self, img):
        img = self.data_transform1(img)
        for t in self.data_transform2:
            img = t(img)
        return img

def get_dataloader(df, batch_size, shuffle, dense_feature_cols):
    ds = PawpularDataset(df['path'].values, df[dense_feature_cols].values, df['norm_score'].values, transform = ImageTransform())
    dl = DataLoader(ds, batch_size=batch_size, shuffle=shuffle, num_workers=8)
    return dl

def get_learning_data(train_df, fold, dense_feature_cols):
    t_df = train_df[train_df['fold'] != fold].copy().reset_index(drop=True)
    v_df = train_df[train_df['fold'] == fold].copy().reset_index(drop=True)

    train_dataloader = get_dataloader(t_df, batch_size=BATCH_SIZE, shuffle=True, dense_feature_cols=dense_feature_cols)
    valid_dataloader = get_dataloader(v_df, batch_size=BATCH_SIZE, shuffle=False, dense_feature_cols=dense_feature_cols)

    dls = DataLoaders(train_dataloader, valid_dataloader)
    
    return dls

class PawpularFigureModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model(PRETRAINED, pretrained=True, num_classes=1)
        
    def forward(self, xin):
        (image, _features) = xin
        return self.model(image)


class PawpularDummyModel(nn.Module):
    def __init__(self, hdim1=32, hdim2=16, dense_feature_cols=DENSE_FEATURE_COLS):
        super().__init__()
        self.model = timm.create_model(PRETRAINED, pretrained=True, num_classes=hdim1)
        self.dense1 = nn.Linear(hdim1, hdim2)
        self.dense2 = nn.Linear(hdim2, 1)

    def forward(self, image, features):
        x = self.model(image)
        x = F.relu(self.dense1(x))
        x = self.dense2(x)
        return x



class PawpularDenseModel(nn.Module):
    def __init__(self, hdim1=32, hdim2=16, dense_feature_cols=DENSE_FEATURE_COLS):
        super().__init__()
        self.model = timm.create_model(PRETRAINED, pretrained=True, num_classes=hdim1)
        self.dense1 = nn.Linear(hdim1+len(dense_feature_cols), hdim2)
        self.dense2 = nn.Linear(hdim2, 1)

    def forward(self, xin):
        (image, features) = xin
        x = self.model(image)
        x = torch.cat([x, features], dim=1)
        x = F.relu(self.dense1(x))
        x = self.dense2(x)
        return x

class PawpularResModel(nn.Module):
    def __init__(self, hdim1=32, hdim2=16, dense_feature_cols=DENSE_FEATURE_COLS):
        super().__init__()
        self.model = timm.create_model(PRETRAINED, pretrained=True, num_classes=1+hdim1)
        self.dense1 = nn.Linear(hdim1+len(dense_feature_cols), hdim2)
        self.dense2 = nn.Linear(hdim2, 1)

    def forward(self, image, features):
        x = self.model(image)
        
        x0 = x[:,0:1]
        x1 = x[:,1:]
        x1 = torch.cat([x1, features], dim=1)
        x1 = F.relu(self.dense1(x1))
        x1 = self.dense2(x1)
        return x0 + x1

class PawpularDotModel(nn.Module):
    def __init__(self, hdim=16, dense_feature_cols=DENSE_FEATURE_COLS):
        super().__init__()
        self.hdim = hdim
        self.model = timm.create_model(PRETRAINED, pretrained=True, num_classes=hdim)
        self.dense = nn.Linear(len(dense_feature_cols), hdim)

    def forward(self, image, features):
        x = self.model(image)
        f = self.dense(features)

        dot_prod = torch.bmm(x.view(-1,1,self.hdim), f.view(-1,self.hdim,1))
        return dot_prod.view(-1,1)

class MSELoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input, target):
        loss = F.mse_loss(torch.sigmoid(input.flatten()), target)
        return loss

def petfinder_mse(input, target):
    return F.mse_loss(torch.sigmoid(input.flatten())*100, target*100)

def get_learner(train_df, fold_num, dense_feature_cols):
    data = get_learning_data(train_df, fold_num, dense_feature_cols)

    if MODEL_INPUT == 'f':
        model = PawpularFigureModel()
    else:
        if MODEL_HEAD == 'dummy':
            model = PawpularDummyModel(dense_feature_cols=dense_feature_cols)
        elif MODEL_HEAD == 'dense':
            model = PawpularDenseModel(dense_feature_cols=dense_feature_cols)
        elif MODEL_HEAD == 'res':
            model = PawpularResModel(dense_feature_cols=dense_feature_cols)
        elif MODEL_HEAD == 'dot':
            model = PawpularDotModel(dense_feature_cols=dense_feature_cols)

    if LOSS_TYPE == 'bce':
        learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_mse).to_fp16()
    elif LOSS_TYPE == 'mse':
        learn = Learner(data, model, loss_func=MSELoss(), metrics=petfinder_mse).to_fp16()

    return learn

def calc_pred(model, dl, n):
    all_preds = []
    model.eval()
    with torch.no_grad():
        for i in range(n):
            preds = None
            for x, _ in dl:
                output = model((x[0].to(device), x[1].to(device)))
                preds = output if preds is None else torch.cat([preds, output], dim=0)
            preds = torch.sigmoid(preds.flatten())
            all_preds.append(preds.cpu().numpy())
    ret = np.mean(np.stack(all_preds), axis=0)
    return ret
            

def main_process():
    train_df = get_train_df(DATASET_PATH, N_FOLDS)

    test_df = get_test_df(DATASET_PATH)
    test_dl = get_dataloader(test_df, batch_size=BATCH_SIZE, shuffle=False, dense_feature_cols=DENSE_FEATURE_COLS)

    #learn = get_learner(train_df, 0, DENSE_FEATURE_COLS)
    #learn.to(device)
    #print(learn.summary())
    #print(learn.lr_find(end_lr=3e-2))

    #exit(0)

    #learn = get_learner(train_df, 0, DENSE_FEATURE_COLS)
    #learn.to(device)
    #print(learn.lr_find(end_lr=3e-2))
    #del learn

    all_preds = []
    for i in range(N_FOLDS):
        print(f'Fold {i} results')
        learn = get_learner(train_df, i, DENSE_FEATURE_COLS)
        learn.model.load_state_dict(torch.load(f'../input/fohe-model0/model_{MODEL_INPUT}_{LOSS_TYPE}_fold_{i}.pth'))
        learn.model.to(device)
        #learn.to(device)
        #learn.fit_one_cycle(5, 2e-5, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=2)])
        
        #learn.fit_one_cycle(5, 2e-5, cbs=[SaveModelCallback(monitor='petfinder_mse', comp=np.less), EarlyStoppingCallback(monitor='petfinder_mse', comp=np.less, patience=2)])
        #learn.recorder.plot_loss()

        #preds, _ = learn.tta(dl=test_dls, n=5, beta=0)
        preds = calc_pred(learn.model.to(device), test_dl, 5)
        all_preds.append(preds)

        #torch.save(learn.model.to('cpu').state_dict(), f'model_{MODEL_INPUT}_{LOSS_TYPE}_fold_{i}.pth')
    
        del learn
        torch.cuda.empty_cache()
        gc.collect()
        
    preds = np.mean(np.stack(all_preds), axis=0)
    return preds


In [ ]:
preds3 = main_process()
print(preds3)

In [ ]:
preds = np.sum(np.stack([preds1*0.585, preds2*0.415]), axis=0)
preds = np.sum(np.stack([preds*0.9, preds3*0.1]), axis=0)
dataset_path = Path('../input/petfinder-pawpularity-score/')
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
sample_df['Pawpularity'] = preds*100
sample_df.to_csv('submission.csv',index=False)